In [ ]:
!pip install bert-extractive-summarizer;

!pip install -U spacy
!pip install transformers # > 4.0.0
!pip install neuralcoref

!python -m spacy download en_core_web_md

import pandas as pd
import numpy as np
import re
from summarizer import Summarizer;

In [2]:
model = Summarizer();

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [4]:
from google.colab import drive 
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/MyDrive/datax/")
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
01_reviews_filtering.ipynb  labeled			   sum_review_bus.csv
austin_rest.csv		    rest_good_bad_review.csv	   zero_shot_test
austin_review.csv	    rest_with_good_bad_review.csv
example.csv		    summarizer


In [5]:
rev_df = pd.read_csv("austin_review.csv")

In [6]:
os.chdir("/content/drive/MyDrive/datax/zero_shot_test")
!ls

all_labelled_w_stars.xlsx
bus_with_label_perc.csv
zero_shot_model_test_02_picking_a_threshold.ipynb
zero_shot_model_test_03_apply_labels_on_rev.ipynb
zero_shot_model_test_04_picking_threshold_for_all.ipynb
zero_shot_model_test.ipynb


In [10]:
bus_df = pd.read_csv("bus_with_label_perc.csv")

In [11]:
rev_df = rev_df[rev_df.business_id.isin(bus_df.business_id)]

In [12]:
bus_df = bus_df.set_index("business_id")

In [13]:
rev_df.shape

(301667, 9)

In [14]:
rev_df_good = rev_df[rev_df.stars >= 4]
rev_df_bad = rev_df[rev_df.stars < 4]

In [15]:
## Collapse reviews by bid with at most 70.
rev_df_good_clps = rev_df_good.groupby("business_id").apply(
    lambda rest_df: rest_df.sample(min(70, rest_df.shape[0])).text.sum()
    )
rev_df_bad_clps = rev_df_bad.groupby("business_id").apply(
    lambda rest_df: rest_df.sample(min(70, rest_df.shape[0])).text.sum()
    )

In [16]:
bus_df["good_reviews"] = rev_df_good_clps
bus_df["bad_reviews"] = rev_df_bad_clps

In [17]:
bus_drop_na_df = bus_df.loc[(bus_df['good_reviews'].notna()) & (bus_df['bad_reviews'].notna())]
## REMEMBER to fillNA at the end with "No enough reviews for this section"

In [29]:
bus_df.shape, bus_drop_na_df.shape

((2119, 20), (2094, 20))

In [64]:
%%time
test_result = model(bus_drop_na_df.good_reviews[0], num_sentences=6)

CPU times: user 9.74 s, sys: 3.91 s, total: 13.7 s
Wall time: 8.99 s


In [65]:
print(test_result)

Live music on Saturday was a nice touchSo yummy! Everything we ordered was a hit including the volcano (came lit on fire), the dynamite and the spicy crab rolls. I've never had the other food offerings (given that the sushi is to die for), but my friend has, and he says it's delish as well. Salmon sashimi for the price was fantastic, very thick slices. Wine was pretty affordable and they told us after the grand opening this weekend they will implement happy hour. It is delightful to have alternatives (like the Kobe burger with chips) for friends who aren't sushi fans. Their sushi is truly some of the best I've had in a while ~ do yourself a favor and order the flaming tiger roll.


In [67]:
## apply Summ on reviews, will take ~10 hours :)

good_summary = [model(rev, num_sentences=6) for rev in bus_drop_na_df.good_reviews]
bus_drop_na_df["good_sum"] = good_summary

bad_summary = [model(rev, num_sentences=6) for rev in bus_drop_na_df.bad_reviews]
bus_drop_na_df["bad_sum"] = bad_summary

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [68]:
bus_drop_na_df.to_csv("sum_review_bus_02.csv") ## please don't set index=False, as the index is bid currently

## AGAIN, fillNA!!!!!!

In [20]:
os.chdir("/content/drive/MyDrive/datax/summarizer")
!ls

Model_08_TextSum.ipynb	Model_v05_summ.ipynb  sum_review_bus_02.csv


In [21]:
bus_w_summarized_rev = pd.read_csv("sum_review_bus_02.csv")

In [25]:
bus_w_summarized_rev.shape

(2094, 22)

In [23]:
os.chdir("/content/drive/MyDrive/datax/zero_shot_test")
!ls

all_labelled_w_stars.xlsx
bus_with_label_perc.csv
zero_shot_model_test_02_picking_a_threshold.ipynb
zero_shot_model_test_03_apply_labels_on_rev.ipynb
zero_shot_model_test_04_picking_threshold_for_all.ipynb
zero_shot_model_test.ipynb


In [24]:
bus_df_02 = pd.read_csv("bus_with_label_perc.csv")
bus_df_02.shape

(2119, 19)

In [42]:
bus_drop_na_df.reset_index(inplace=True)

In [43]:
bus_w_summarized_rev["business_id"] = bus_drop_na_df.business_id

In [45]:
## move bid to front
col = bus_w_summarized_rev.pop("business_id")
bus_w_summarized_rev.insert(0, col.name, col)

In [53]:
bus_w_summarized_rev.shape, bus_drop_na_df.shape

((2094, 23), (2094, 21))

In [55]:
all(bus_drop_na_df.business_id == bus_w_summarized_rev.business_id)

True

In [58]:
merged = bus_df_02.merge(bus_w_summarized_rev[["business_id", "good_sum", "bad_sum"]], how='left', on='business_id')

In [61]:
merged.isna().any()

business_id             False
name                    False
address                  True
city                    False
state                   False
postal_code             False
latitude                False
longitude               False
stars                   False
review_count            False
is_open                 False
attributes               True
categories              False
hours                    True
expensive               False
bad_customer_service    False
poor_ambience           False
small_portion_size      False
bad_food_taste          False
good_sum                 True
bad_sum                  True
dtype: bool

In [62]:
merged.fillna(value={"good_sum": "not enough reviews", "bad_sum": "not enough reviews"}, inplace=True)

In [64]:
merged.isna().any()

business_id             False
name                    False
address                  True
city                    False
state                   False
postal_code             False
latitude                False
longitude               False
stars                   False
review_count            False
is_open                 False
attributes               True
categories              False
hours                    True
expensive               False
bad_customer_service    False
poor_ambience           False
small_portion_size      False
bad_food_taste          False
good_sum                False
bad_sum                 False
dtype: bool

In [65]:
merged.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,expensive,bad_customer_service,poor_ambience,small_portion_size,bad_food_taste,good_sum,bad_sum
0,NRPemqVb4qpWFF0Avq_6OQ,Eurasia Sushi Bar & Seafood,"7101 W Hwy 71, Ste C-13",Austin,TX,78735.0,30.234533,-97.877262,4.5,395,1,"{'Ambience': ""{'touristy': False, 'hipster': F...","Bars, Nightlife, Cocktail Bars, Seafood, Resta...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-22:0', ...",0.050847,0.070218,0.033898,0.000000,0.067797,Live music on Saturday was a nice touchSo yumm...,I've had very cheap to very expensive sushi in...
1,bRsDZ44CD3uhGnRY3NeQhQ,Wendy's,6247 Mcneil Drive,Austin,TX,78729.0,30.441875,-97.746581,2.0,46,1,"{'RestaurantsPriceRange2': '1', 'OutdoorSeatin...","Fast Food, Restaurants, Burgers","{'Monday': '6:30-1:0', 'Tuesday': '6:30-1:0', ...",0.000000,0.586957,0.347826,0.000000,0.195652,In the last few months I've never had any bad ...,i. come and i have been here (sadly) about 20x...
2,Pk4ZwXwUU50BDn5gqw_rKg,Johnny Carino's,9500 S IH-35 Service Rd,Austin,TX,78748.0,30.162081,-97.789132,3.0,136,1,"{'RestaurantsGoodForGroups': 'True', 'Business...","Italian, Salad, Pizza, Nightlife, Restaurants,...","{'Monday': '11:0-21:30', 'Tuesday': '11:0-21:3...",0.097902,0.279720,0.111888,0.000000,0.272727,Came in with my girlfriend the food was ok but...,We came here for dinner b/c we thought we knew...
3,IFB2K3BEZ2L_Mv5AbUD26Q,Chispas,214 W 4th St,Austin,TX,78701.0,30.266996,-97.745362,3.5,119,1,"{'RestaurantsTakeOut': 'True', 'OutdoorSeating...","Tex-Mex, Mexican, Tacos, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",0.066116,0.132231,0.090909,0.016529,0.314050,What a hidden gem... $4 tacos and AMAZING sals...,"Server was friendly, just no utensils for a wh..."
4,fBvQQUk1-V9IUA8PbuYFWg,Newk's Eatery,408 Congress Ave,Austin,TX,78701.0,30.266639,-97.743659,3.5,43,1,"{'HasTV': 'True', 'BusinessParking': ""{'garage...","Pizza, Restaurants, American (Traditional), Am...","{'Monday': '0:0-0:0', 'Tuesday': '10:30-18:0',...",0.116279,0.139535,0.069767,0.023256,0.093023,Newk's gets the 5 star for the freshness of ev...,"The food is pretty good, with quite a bit of f..."


In [66]:
os.chdir("/content/drive/MyDrive/datax/")
merged.to_csv("rest_w_summarizer_and_zero_shot_results.csv", index=False)